# **En este algoritmo de regresion logistica logística “one-vs-all” se realizara la predición de enfermedades en las hojas de arboles de mango.**

- Cristian Ubaldo Tirado Murcia

#**Importar librerias**

In [ ]:
import os
import random
import cv2 as cv
import numpy as np
import tensorflow as tf
import tensorflow_hub as hub
from skimage.io import imsave
from skimage.io import imread
import matplotlib.pyplot as plt
from tensorflow.keras import layers
from skimage.transform import resize
from tensorflow.keras import Sequential
from tensorflow.keras.layers import add,Dense
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import image_dataset_from_directory
from sklearn.metrics import accuracy_score, classification_report


# **Folders explorer**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
folder = '/content/drive/MyDrive/Computervision/RegresiónLogistica/mango_leaf_disease'

In [ ]:
def count_images(folder):
    # Acceder a las subrutas, contar el numero de archivos de extensión de imagen en cada uno
    subfolders = [f.path for f in os.scandir(folder) if f.is_dir()]
    for subfolder in subfolders:
        files = os.listdir(subfolder)
        image_count = sum(1 for file in files if file.lower().endswith(('.png', '.jpg', '.jpeg', '.gif', '.bmp')))
        subfolder_name = os.path.basename(subfolder)
        print(f"En la carpeta '{subfolder_name}' hay {image_count} imágenes.")

# Llamar a la función para contar imágenes en la carpeta principal
count_images(folder)

En la carpeta 'Anthracnose' hay 500 imágenes.
En la carpeta 'Bacterial Canker' hay 500 imágenes.
En la carpeta 'Cutting Weevil' hay 500 imágenes.
En la carpeta 'Die Back' hay 500 imágenes.
En la carpeta 'Gall Midge' hay 500 imágenes.
En la carpeta 'Healthy' hay 500 imágenes.
En la carpeta 'Powdery Mildew' hay 500 imágenes.
En la carpeta 'Sooty Mould' hay 500 imágenes.


In [ ]:
#solo correr una vez
'''def rename_files(folder):
    # Obtener la lista de archivos en la carpeta
    files = os.listdir(folder)
    # Obtener el número total de archivos para calcular la longitud del número
    total_files = len(files)
    num_digits = len(str(total_files))
    # Iterar sobre los archivos y renombrarlos
    for i, filename in enumerate(files, 1):
        # Construir el nuevo nombre de archivo con ceros a la izquierda
        new_filename = str(i).zfill(num_digits) + os.path.splitext(filename)[1]
        # Construir la ruta completa del archivo antiguo y nuevo
        old_path = os.path.join(folder, filename)
        new_path = os.path.join(folder, new_filename)
        # Renombrar el archivo
        os.rename(old_path, new_path)
        print(f"Renombrando {filename} a {new_filename}")

def recursive(root_folder):
    # Iterar sobre todas las carpetas y subcarpetas
    for root, dirs, files in os.walk(root_folder):
        for folder in dirs:
            folder_path = os.path.join(root, folder)
            rename_files(folder_path)

# Ejemplo de uso: Renombrar archivos en una carpeta y subcarpetas
recursive(folder)
'''

Renombrando 20211008_124302 (Custom).jpg a 001.jpg
Renombrando 20211008_124252 (Custom).jpg a 002.jpg
Renombrando 20211008_124249 (Custom).jpg a 003.jpg
Renombrando 20211008_124301 (Custom).jpg a 004.jpg
Renombrando 20211008_124250 (Custom).jpg a 005.jpg
Renombrando 20211008_124256 (Custom).jpg a 006.jpg
Renombrando 20211008_124253 (Custom).jpg a 007.jpg
Renombrando 20211011_133809 (Custom).jpg a 008.jpg
Renombrando 20211011_133430 (Custom).jpg a 009.jpg
Renombrando 20211011_133423 (Custom).jpg a 010.jpg
Renombrando 20211011_133407 (Custom).jpg a 011.jpg
Renombrando 20211011_133406 (Custom).jpg a 012.jpg
Renombrando 20211011_133205 (Custom).jpg a 013.jpg
Renombrando 20211011_133207 (Custom).jpg a 014.jpg
Renombrando 20211011_133419 (Custom).jpg a 015.jpg
Renombrando 20211011_133201 (Custom).jpg a 016.jpg
Renombrando 20211011_133203 (Custom).jpg a 017.jpg
Renombrando 20211011_133200 (Custom).jpg a 018.jpg
Renombrando 20211011_131259 (Custom).jpg a 019.jpg
Renombrando 20211011_133412 (Cu

# **Lowest images size per folder**


In [ ]:
def smallest_image(folder):
    # Obtener todas las subcarpetas dentro de la carpeta principal
    subfolders = [f.path for f in os.scandir(folder) if f.is_dir()]

    # Recorrer cada subcarpeta
    for subfolder in subfolders:
        # Obtener la lista de archivos en la subcarpeta
        files = os.listdir(subfolder)
        smallest_image = None
        smallest_dimensions = float('inf')

        # Buscar la imagen de menor dimensión en la subcarpeta
        for file in files:
            if file.lower().endswith(('.png', '.jpg', '.jpeg', '.gif', '.bmp')):
                image_path = os.path.join(subfolder, file)
                # Leer la imagen
                image = cv.imread(image_path)
                height, width, _ = image.shape
                # Obtener las dimensiones de la imagen
                dimensions = f"{width}*{height}"
                # Actualizar la imagen de menor dimensión si es más pequeña que la actual
                if dimensions < str(smallest_dimensions):
                     smallest_dimensions = dimensions
                     smallest_image = image_path
        # Imprimir el nombre de la subcarpeta y la ruta de la imagen de menor dimensión
        subfolder_name = os.path.basename(subfolder)
        print(f"Lowest at '{subfolder_name}' is: {smallest_image[-7:]} : {smallest_dimensions}")

In [ ]:
# Ejemplo de uso: Encontrar la imagen más pequeña en una carpeta y subcarpetas
smallest_image(folder)

Lowest at 'Anthracnose' is: 008.jpg : 240*240
Lowest at 'Bacterial Canker' is: 001.jpg : 240*320
Lowest at 'Cutting Weevil' is: 001.jpg : 240*240
Lowest at 'Die Back' is: 001.jpg : 240*240
Lowest at 'Gall Midge' is: 227.jpg : 187*320
Lowest at 'Healthy' is: 415.jpg : 210*320
Lowest at 'Powdery Mildew' is: 202.jpg : 189*320
Lowest at 'Sooty Mould' is: 001.jpg : 240*320


In [ ]:
# Definir las clases
clases = ['Sooty Mould', 'Powdery Mildew', 'Healthy', 'Gall Midge', 'Die Back', 'Cutting Weevil','Bacterial Canker', 'Anthracnose']

In [ ]:
X = []
y = []

In [ ]:
for label, clase in enumerate(clases):
    carpeta = os.path.join(ruta_directorio, clase)
    for imagen_nombre in os.listdir(carpeta):
        imagen_ruta = os.path.join(carpeta, imagen_nombre)
        # Verificar si la ruta es un archivo de imagen
        if os.path.isfile(imagen_ruta) and imagen_nombre.lower().endswith(('.png', '.jpg', '.jpeg', '.gif', '.bmp')):
            imagen = imread(imagen_ruta)
            imagen_redimensionada = resize(imagen, (128, 128, 3), anti_aliasing=True)
            X.append(imagen_redimensionada.flatten())
            y.append(label)

X = np.array(X)
y = np.array(y)

In [ ]:
X

array([[0.87585543, 0.88369856, 0.87193386, ..., 0.92210779, 0.92210779,
        0.91426465],
       [0.9199052 , 0.92774834, 0.91598363, ..., 0.81973236, 0.8158108 ,
        0.79620295],
       [0.89816137, 0.89816137, 0.89031823, ..., 0.79711856, 0.78927542,
        0.74221659],
       ...,
       [0.87270727, 0.88839355, 0.89231512, ..., 0.64794041, 0.66752759,
        0.65190331],
       [0.84871929, 0.85264086, 0.860484  , ..., 0.56306321, 0.58267105,
        0.59443575],
       [0.79141397, 0.81102181, 0.82670809, ..., 0.70023847, 0.6963169 ,
        0.71592474]])

In [ ]:
y

array([0, 0, 0, ..., 7, 7, 7])

In [ ]:
# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15, random_state=42)

In [ ]:
# Crear una lista para almacenar los modelos de regresión logística
models = []

# Entrenar un modelo para cada clase utilizando el enfoque "one-vs-all"
for class_label in np.unique(y_train):
    # Crear un clasificador de regresión logística
    model = LogisticRegression(solver='liblinear')

    # Convertir las etiquetas a binarias para la clase actual vs. las demás clases
    y_train_binary = np.where(y_train == class_label, 1, 0)

    # Entrenar el modelo
    model.fit(X_train, y_train_binary)

    # Agregar el modelo entrenado a la lista
    models.append(model)

In [ ]:
models

[LogisticRegression(solver='liblinear'),
 LogisticRegression(solver='liblinear'),
 LogisticRegression(solver='liblinear'),
 LogisticRegression(solver='liblinear'),
 LogisticRegression(solver='liblinear'),
 LogisticRegression(solver='liblinear'),
 LogisticRegression(solver='liblinear'),
 LogisticRegression(solver='liblinear')]

In [ ]:
# Hacer predicciones en el conjunto de prueba
y_pred = np.zeros((len(X_test), len(models)), dtype=int)
for i, model in enumerate(models):
    y_pred[:, i] = model.predict(X_test)

# Obtener las predicciones finales combinando los resultados de todos los modelos
final_predictions = np.argmax(y_pred, axis=1)

# Calcular la precisión
accuracy = accuracy_score(y_test, final_predictions)
print("Precisión:", accuracy)

# Mostrar un informe de clasificación
print("Informe de clasificación:")
print(classification_report(y_test, final_predictions, target_names=clases))

Precisión: 0.66625
Informe de clasificación:
                  precision    recall  f1-score   support

     Sooty Mould       0.34      0.77      0.47       123
  Powdery Mildew       0.70      0.50      0.59        86
         Healthy       0.91      0.79      0.84       112
      Gall Midge       0.40      0.18      0.25       101
        Die Back       0.99      0.81      0.89       104
  Cutting Weevil       0.94      0.93      0.94        90
Bacterial Canker       0.73      0.59      0.65        91
     Anthracnose       0.99      0.72      0.83        93

        accuracy                           0.67       800
       macro avg       0.75      0.66      0.68       800
    weighted avg       0.74      0.67      0.68       800



In [ ]:
# Definir las clases posibles del entrenamiento
clases = ['']
# Inicializar listas para almacenar las imágenes y las etiquetas
X = []
y = []

In [ ]:
# Leer las imágenes y asignar las etiquetas
for label, clase in enumerate(clases):
    folder = os.path.join(folder, clase)
    for imagen_nombre in os.listdir(folder):
        imagen_ruta = os.path.join(folder, imagen_nombre)
        imagen = imread(imagen_ruta)
        # Redimensionar la imagen a un tamaño común (ajustar según sea necesario)
        imagen_redimensionada = resize(imagen, (128, 128, 3), anti_aliasing=True)
        X.append(imagen_redimensionada.flatten())
        y.append(label)

ValueError: ImageIO does not generally support reading folders. Limited support may be available via specific plugins. Specify the plugin explicitly using the `plugin` kwarg, e.g. `plugin='DICOM'`

In [ ]:
# Convertir las listas en matrices numpy
X = np.array(X)
y = np.array(y)

In [ ]:
# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=108)

# Crear una lista para almacenar los modelos de regresión logística
models = []

# Entrenar un modelo para cada clase utilizando el enfoque "one-vs-all"
for class_label in np.unique(y_train):
    # Crear un clasificador de regresión logística
    model = LogisticRegression(solver='liblinear')

    # Convertir las etiquetas a binarias para la clase actual vs. las demás clases
    y_train_binary = np.where(y_train == class_label, 1, 0)

    # Entrenar el modelo
    model.fit(X_train, y_train_binary)

    # Agregar el modelo entrenado a la lista
    models.append(model)

# Hacer predicciones en el conjunto de prueba
y_pred = np.zeros((len(X_test), len(models)), dtype=int)
for i, model in enumerate(models):
    y_pred[:, i] = model.predict(X_test)

# Obtener las predicciones finales combinando los resultados de todos los modelos
final_predictions = np.argmax(y_pred, axis=1)

# Calcular la precisión
accuracy = accuracy_score(y_test, final_predictions)
print("Precisión:", accuracy)

# Mostrar un informe de clasificación
print("Informe de clasificación:")
print(classification_report(y_test, final_predictions, target_names=clases))


Precisión: 0.665
Informe de clasificación:
                  precision    recall  f1-score   support

     Sooty Mould       0.35      0.74      0.48       123
  Powdery Mildew       0.73      0.64      0.68        86
  Cutting Weevil       0.99      0.95      0.97       112
Bacterial Canker       0.68      0.61      0.65       101
      Gall Midge       0.38      0.21      0.27       104
     Anthracnose       0.93      0.69      0.79        90
         Healthy       0.89      0.73      0.80        91
        Die Back       0.99      0.73      0.84        93

        accuracy                           0.67       800
       macro avg       0.74      0.66      0.68       800
    weighted avg       0.73      0.67      0.68       800

